# Αιτήσεις για Visa 

#### Περίληψη

Θα ερευνήσουμε πώς άλλαξαν οι αιτήσεις για visa στην Αμερική όλα αυτά τα χρόνια. 


#### Πηγή Δεδομένων

Nonimmigrant Visa Issuances by Visa Class and by Nationality, 1999+,
https://travel.state.gov/content/travel/en/legal/visa-law0/visa-statistics/nonimmigrant-visa-statistics.html

#### Αρχεία

- FYs97-16_NIVDetailTable1.xls, "Nonimmigrant Visa Issuances by Visa Class and by Nationality, 1999+"

#### Τι θα μάθουμε:

- Επεξεργασία αρχείων Excel με πολλά φύλλα
- Συνδυασμός πολλών dataframes
- Γέμισμα κενών γραμμών (null values) με τη μέθοδο του γεμίσματος προς τα κάτω (filling down)
- Καθάρισμα, διαγραφή και φιλτάρισμα γραμμών
- Μετονομασία στηλών
- Επεξεργασία γραφικών σε matplotlib

# Εισάγουμε το FYs97-16_NIVDetailTable1.xls, αλλά μόνο για το 1999

In [ ]:
import pandas as pd
%matplotlib inline

## Ας δούμε τις πρώτες γραμμές να σιγουρευτούμε ότι είναι δεδομένα του 1999


In [ ]:
df= pd.read_excel('FYs97-16_NIVDetailTable1.xls', sheetname='FY99') #παρατηρήστε ότι επιλέγω έτσι ποιο φύλλο διαβάζω
df

In [ ]:
df.shape

## Εμφανίζουμε το μέγιστο αριθμό των γραμμών 

**Δεν θέλουμε να έχει `...` στη μέση**.

In [ ]:
pd.set_option("display.max_rows",217)

In [ ]:
df.dtypes

## Αλλάζουμε το "Fiscal Year 1999" με  "Country"

In [ ]:
df.rename(columns = {'Fiscal Year 1999' : 'Country'}, inplace = True)


## Κοιτάμε τις 2 πρώτες γραμμές για να δούμε ότι το μετονομάσαμε σωστά! 


In [ ]:
df.head()

# Καθάρισμα

Συνήθως καθαρίζουμε στήλες τώρα **το πρόβλημα είναι στις γραμμές**.



## Πάμε να βρούμε όλα τα "βρώμικα" rows 

In [ ]:
df[df['Grand Total'].isnull()]

## Υπάρχει κάποια στήλη για την ήπειρο? Πώς ξέρουμε σε ποια ήπειρο είναι η κάθε χώρα? 


## Καθάρισμα

έχουμε διάφορες γραμμές που δεν μας αρέσουν:

- Τίτλοι για ηπείρους ("Africa," "Europe," etc)
- Σύνολα για ηπείρους ("Totals for Africa," "Totals for Europe," etc.)
- Οι κενές γραμμές ανάμεσα στις ηπείρους 

Αλλά θα τις *καθαρίσουμε*! Με **τρία βήματα**:

1. Φτιάχνουμε μια νέα στήλη "continent" για κάθε πρώτη γραμμή στις χώρες 
2. Αντιγράφουμε την τιμή προς τα κάτω σε όλες τις κενές γραμμές
3. Σβήνουμε τις άχρηστες γραμμές

### βήμα 1: Για κάθε γραμμή/τίτλο για continent, προσθέτουμε μια νέα στήλη  `continent` με την τιμή του continent



## βήμα 1 επιβεβαίωση


In [ ]:
df.loc[df['A-1'].isnull(), 'continent'] =  df['Country']
df

## βήμα 2: Γέμισμα

Με τα pandas παίρνουμε το όνομα της ηπείρου και το αντιγράφουμε κάτω στις κενές γραμμές. 

Στην αρχή θα δείχνει έτσι ... 

|Country|Grand Total|Continent|
|---|---|---|
|Africa|NaN|Africa|
|Algeria|6454.0|NaN|
|Angola|2843.0|NaN|
|Asia|NaN|Asia|
|Afghanistan|2217.0|NaN|
|Bahrain|2502.0|NaN|

και μετά κάπως έτσι:

|Country|Grand Total|Continent|
|---|---|---|
|Africa|NaN|Africa|
|Algeria|6454.0|**Africa**|
|Angola|2843.0|**Africa**|
|Asia|NaN|Asia|
|Afghanistan|2217.0|**Asia**|
|Bahrain|2502.0|**Asia**|

In [ ]:
df['continent'].fillna(method='ffill', inplace = True)
df.head()

## βήμα 2 Επιβεβαίωση

τσεκάρετε ότι έχετε για **Europe και Africa  56.**

In [ ]:
df.continent.value_counts()

## βημά 3: διαγράφοντας τις κακές γραμμές

### Διαγραφή της γραμμής "Grand Total"


In [ ]:
new_df = df.dropna(subset=['Grand Total'])
new_df.head()

## Τσεκάρουμε

In [ ]:
new_df.head(55)

### Σβήστε τις γραμμές του "Country" που έχουν "Totals"

Τώρα που έχουμε την στήλη "Continent" μπορούμε να υπολογίσουμε τα σύνολα με το groupby.


In [ ]:
new = new_df[~df.Country.str.contains('Total', na= False)]

In [ ]:
#df[~df.Country.str.contains('Total', na= False)] #df[~df.Country.str.contains('stat$', na= False)]

In [ ]:
new.shape

# Ας φτιάξουμε μερικά γραφικά 



In [ ]:
new.sort_values(by= 'Total Visas').tail(10).plot(x='Country', y='Total Visas', kind='barh', figsize=(5, 10))


## Καθαρίζοντας τα γραφικά!

- Προσθέστε τίτλο
- Προσθέστε label στον άξονα του χ 
- Διώξτε το label στον αριστερό άξονα
- Διώξτε τα πλαϊνά (legend)
- Διώξτε τα κουτιά
- Διώξτε τα τικς (tick marks)
- Προσθέστε κόμματα στους άξονες των αριθμών

Περισσότερα εδώ:

- https://matplotlib.org/gallery.html
- Ειδικά εδώ https://matplotlib.org/examples/ticks_and_spines/spines_demo.html

Για να μορφοποιήσετε τους αριθμούς δοκιμάστε 

```python
import matplotlib.ticker as ticker
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.0f}"))
```

In [ ]:
import matplotlib.ticker as ticker


In [ ]:
ax = new.sort_values(by= 'Total Visas').tail(10).plot(x='Country', y='Total Visas', kind='barh', figsize=(5, 10))
ax.set_title("Total Visas per country")
ax.grid(axis='x')
ax.set_xlabel("Total amount of Visas")
ax.xaxis.set_label("This...")
ax.set_ylabel("")
ax.legend('')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.0f}"))
ax.xaxis.set_ticks_position('none')
ax.yaxis.set_ticks_position('none')

## Φτιάξτε ένα γράφημα για το σύνολο των F-1 visas για κάθε ήπειρο

And make it look nice!

In [ ]:
ax = new[new.continent != 'Unknown' ].groupby('continent')['F-1'].sum().sort_values(ascending = True).plot(kind='barh', color='pink')
ax.set_title("Total Visas per continent")

# Διαβάστε όλα τα δεδομένα από το 2000-2018



In [ ]:
sheetnames = [
    'FY97', 'FY98', 'FY99', 'FY00', 'FY01', 'FY02', 'FY03', 'FY04', 'FY05', 'FY06',
    'FY07', 'FY08', 'FY09', 'FY10', 'FY11', 'FY12', 'FY13', 'FY14', 'FY15', 'FY16', 'FY17', 'FY18'
]

In [ ]:
list_of_shs = [pd.read_excel("FYs97-16_NIVDetailTable1.xls", sheetname=sheet) for sheet in sheetnames]

# Homework in LAB

## Ξανακαθαρίστε όπως κάναμε παραπάνω 

## βήμα 1: Για κάθε dataframe, αλλάξτε το "Fiscal Year XXXX" σε "Country"



```python
df.columns.str.replace('Fiscal Year.*', 'Country')
```

In [ ]:
for sheet in list_of_shs:
    sheet.columns =sheet.columns.str.replace('Fiscal Year.*', 'Country')

In [ ]:
list_of_shs

## βήμα 2: Προσθέστε την στήλη  'Continent' και πετάξτε τις κακές γραμμές

Ο παρακάτω κώδικας θα διαγράψει όλες τις στήλες στις οποίες το 'Country' έχει μέσα 'Totals for'. Το κόλπο που χρησιμοποιήσαμε παραπάνω δεν θα δουλέψει γιατί τώρα έχουμε ολόκληρη λίστα. 

```python
df.drop(df.index[df['Country'].str.contains("Totals for", na=False)])
```

In [ ]:
import numpy as np
for sheet in list_of_shs:
    df.continent = np.nan
    sheet.loc[sheet['Grand Total'].isnull(), 'continent'] =  sheet['Country']   
    sheet['continent'].fillna(method='ffill', inplace = True)
    sheet = sheet.drop(sheet.index[sheet['Country'].str.contains("Totals for", na=False)])


In [ ]:
list_of_shs[0]

## βήμα 3: Προσθέστε τη χρονιά σε κάθε dataframe

Δεν μπορούμε ακόμη να τα συνδυάσουμε γιατί προς το παρόν τα θέλουμε ξεχωριστά. Χρησιμοποιήστε τον παρακάτω κώδικα για να πλησιάσετε σε αυτό που θέλετε.  

```python
for sheet_df, fiscal_year in zip(sheets, sheetnames):
    sheet_df['year'] = fiscal_year
```

In [ ]:
for sheet_df, fiscal_year in zip(list_of_shs, sheetnames):
    sheet_df['year'] = fiscal_year

In [ ]:
list_of_shs

# Ενώστε τα dataframes σε ένα μεγάλο dataframe

Ενώστε τα και τσεκάρετε ότι έχετε 4464 γραμμές και 96 στήλες

- Συμβουλή: Μην χρησιμοποιήσετε .merge!

In [ ]:
combined = pd.concat(list_of_shs, ignore_index=True)

In [ ]:
combined.dtypes

## Λίγο καθάρισμα

Αλλάξτε το FY97 σε 1997, FY98 σε 1998, FY99 σε FY1999, FY00 σε 2000, κ.λπ.



In [ ]:
combined['year']= combined['year'].str.replace('FY9', '199')

In [ ]:
combined['year']= combined['year'].str.replace('FY0', '200')

In [ ]:
combined['year']= combined['year'].str.replace('FY1', '201')

In [ ]:
combined

# Μερικά γραφήματα

## Κάντε ένα γράφημα για τις βίζες που δόθηκαν στην Κολομβία όλα αυτά τα χρόνια. 

Κάντε τα όμορφα όπως δείξαμε παραπάνω!

In [ ]:
ax = combined[combined.Country == 'Colombia' ].groupby('year')['Total Visas'].sum().plot(kind='barh', color='black')
ax.set_title("Colombia")
ax.grid(axis='x')
ax.set_xlabel("Total amount of Visas")
ax.xaxis.set_label("This...")
ax.set_ylabel("")
ax.legend('')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.0f}"))
ax.xaxis.set_ticks_position('none')
ax.yaxis.set_ticks_position('none')

## Φτιάξτε ένα γράφημα για τις βίζες που δόθηκαν στο Ιρακ

In [ ]:
ax = combined[combined.Country == 'Iraq' ].groupby('year')['Total Visas'].sum().sort_values(ascending = True).plot(kind='barh', color='yellow')
ax.set_title("Iraq")
ax.grid(axis='x')
ax.set_xlabel("Total amount of Visas")
ax.xaxis.set_label("This...")
ax.set_ylabel("")
ax.legend('')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.0f}"))
ax.xaxis.set_ticks_position('none')
ax.yaxis.set_ticks_position('none')

## Φτιάξτε ένα γράφημα για τις βίζες που δόθηκαν στο Μεξικό

In [ ]:
ax = combined[combined.Country == 'Mexico' ].groupby('year')['Total Visas'].sum().sort_values(ascending = True).plot(kind='bar', color='green')
ax.set_title("Mexico")
ax.set_xlabel("Year")
ax.xaxis.set_label("Year")
ax.set_ylabel("")
ax.legend('')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.xaxis.set_ticks_position('none')
ax.yaxis.set_ticks_position('none')

## Φτιάξτε άλλα 2 γραφήματα της επιλογής σας!